In [19]:
import os
import time
import re
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# CSV 파일 경로
file_path = "kakao_작가 작품 수, 원작 여부 병합 전 최종_only19.csv"
df = pd.read_csv(file_path, encoding="utf-8-sig")
title_ids = df['title_id'].tolist()

total = len(title_ids)
chunk_size = 100

output_folder = "kakao_novel_original_under19_chunks"
os.makedirs(output_folder, exist_ok=True)

options = Options()
options.add_argument("--headless=new")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# 로그인에 필요한 최소 쿠키만 넣기 (주신 쿠키 값에서 domain과 name 기준 최소 추출)
login_cookies =  [
    {"name": "__T_", "value": "1", "domain": ".page.kakao.com", "path": "/", "httpOnly": False, "secure": False},
    {"name": "__T_SECURE", "value": "1", "domain": ".page.kakao.com", "path": "/", "httpOnly": False, "secure": True},
    {"name": "_kahai", "value": "f341a75bb3cd3a4bd1a22bab3aceeb6e20dfb5e2ace653fc4cca086b03fd53b1", "domain": ".kakao.com", "path": "/", "httpOnly": True, "secure": True},
    {"name": "_karmt", "value": "xLA4IXI9utnZaXTybiZ2RQDpRJmqUt7W1ACIPMdXpxO-VL701N0DKL24yBpXo-dK", "domain": ".kakao.com", "path": "/", "httpOnly": True, "secure": True},
    {"name": "_kau", "value": "73106297ba50319faabb0d29da81edf43b0a3662a795a16010d184f71adb834de7f1d57d56c56f5c593a57828709bdc6a482275b137e168115a16dd2bc1ea49be71ed11d6e6f8ea53c61d6d984f35764456253bf49042401a361b814b2bdc8225bf2cf7ec338494b2ad7b0e8f8d4e95b8550eea4e1d2f519e79880b135313731343732363033343632303736373632343139323030343337303735335f92c5aed2e435d9620a7f08bf3e985e", "domain": ".kakao.com", "path": "/", "httpOnly": True, "secure": True},
    {"name": "_kawlt", "value": "XYpEZj6ZSykdch9_siigDwxoa1PFwxSWkni8271CL8Nq9baBnf5Zx7TR8cmvcnQ_qLY-i84QIYzHyhBFWYJ3jLHokhfIfvGvt4Beh9Y6WhDdSrtnBnXLkPEesLIpPhbA", "domain": ".kakao.com", "path": "/", "httpOnly": True, "secure": True},
    {"name": "_kpawbat", "value": "ubIFO1skG3pu6HGZZ03pIDo5sAkn1WFYAAAAAQoXIiAAAAGXQI8-9ejqOP6o1CZo", "domain": ".kakao.com", "path": "/", "httpOnly": True, "secure": True},
    {"name": "_kpwtkn", "value": "U2FsdGVkX19Ta%2BP0NdaLaal%2BwgqXZCx%2FTtzuwUFZqhpG9ebGdNJaDfmwi6TFPbQLZnrIaAVDs%2BNNxRoY%2Bcw9aq5QHjIqS62%2FSc6ZLCMHNxhXlliwIDHbcXVGkIS7T024SpcBdlq9vdXSQPFvU3n9ZdGn0s3lBGIUIvs0KewWtqOolM6nUVlTHyNqdkhazh1wG09MhZgupFNlSkDBW3P0xmvwnGCd00VgwqHidB5GNPeeKnPNMb1z7jo1dX2NUUzlJLL35ZkMg99w3X69Fuv6ShTBUeTFjK7ktczOQr%2BAl3rIarrvfg%2FopY2l5UXyq%2FBOBE%2BgA1fVoXYTrxsdmuphGLo4VK1FEIjFvWYIbJo6eRy7%2Fl85cG8gQy4DMUbiMQBYedAlb0N8e9Xn6KcXZmnsDOGjg7Y5JjbYLLVjYhhd13osgjU1JSkjtb0CPlcew6wiQEE73%2BZ0BLl7%2B46WgB1lCANfQTqFmJgQfKbzfqn3Pn%2BDvENjx3EJeOylxUUg08KNK0A7OSyVk3%2BtS%2BiFgTkYJzlYOM30KMgFp13wzv1IBfmQgN12ZIxJnUrqNXgHgIshQS2E0K4shOMjiggMzxyZ8Tw0Wr4TpViobxO9MUUPo5XmXNBEKqOarOic%2BVGZjgvqSRWAB8pM%2Fxq1DjUkeD%2BqBOhrX2unh6fslTS5uEBMWka%2Fp%2BcpIdX7qb3Kso8AafxV9gbj5QM3qajZ3q58gh%2BfH2ZzRI6JwXIW7x%2FLU2QUFl2VsjFcLRXEwtVBxLrtQLnpXIDYsEcpzRaRuTarkfB3f1v%2B7SJOOpEYxAz41XqcTCeOtya0Tl%2FrEFVhS3SlldPRdF6rk5Ip5sCI2d2J%2Bl%2Fqhq6Qk1Ltlq%2BKgOxVTmlQVuWhzEpsOYE1YRrnZeL1S4Tul3uQzLH0bGEPLNXl%2F8NLDwiSZY7AOeWbXPT%2F9bXhDpuCqpzIfkHOBEYANZkwgb3RMmC92K1fPa1fMyNCw51ZzZc5ypB0U%2B3xZiFPR68%2BypU0sihb4zmtt%2FyW7LO0DQQkYYToTjlwElmvWhxFvIbN8BCG843LyIQbtGMvzpkvvM3XLeUWBHja9OcyN%2FFnSL0AHAK4dH2%2FxnYXwdHIkwjDk%2F7aDWbVr5MX5QI3ykhFW%2FnhvhDu5cZFYNilkx2PYtWqQW958kb98PJe83lRvDiLZ%2B4sWUiqcRuOAQCfDXWp3K4aLJnYiaeOfCo3yBO1JLRirFhVT91LesrJ4dzbV505ndm2ZutdvB%2FQUzUJGq40I0VELiROGfud2FJo%2B%2FYpnGa0NSOJRkcaFio3bz0w%2FQNPY%2BmhDiBBEo%2F7FUnanzcq%2F1L08nCrQG50ueKvW2zY7Xc4DGhjFXpp3zY65Q3vgKSv8a5BcsjIYkO9mtdOR4%2FLRu3%2FcNGMQrLyNmRVvbOLkmr%2FNY6n1jCji3%2FFVWhtMlg2LF0Jd4Xjisyi9bR9OmqabxL9LuqZXQnLRTvh9nomIRfcC6E9xzKgKGAlZMPsOX%2FWziW50hpgMpOS4%2FfCw7piY4ApMeiH0S8W%2BLUwMKZ6GRtWqzGiQnoEsj%2FlfnR3ztfpxA9U7nZARyHlwe4%2Fp9%2BZAlGTYVPa6xML2%2FnnvyMM9PZd8%2FfEzSf9u5B12Fp7Ts2F7ltbnJSekrc7Vwmg8EEP%2FOGTNz9MUEGhRzs7dpN8VTPijbFM6s5CI4T%2B2nUniMYnDKw1pqKJwKIhOes2f9kU%2Bgv7hIB2E4DNxntsX9Ys7ciTClFGSI8vZKi%2FR1mt9t42Nds8Yn7DgkuPsovdQIFLAiVLbfGeR1XVagPHseTGxWk0KH1a9XBUnuZ5FGYUmlk8NDuVbyPSP23PUlsuouy%2FrU7z7G19d2hcunkJFfLviLR%2FCo1cfQWF93UE%2FmWegAG5q3LqTrlzW0up%2BtuW8vw9ieFpjMp%2Bo9QzAKrOTijlDICOaFxolKiaTD6%2F93JWz3BuVdUruHDmWyBcLeBqHZ0IjPU%3D", "domain": ".kakao.com", "path": "/", "httpOnly": True, "secure": True},
]

driver.get("https://page.kakao.com/")
time.sleep(5)  # 페이지 로딩 대기

for cookie in login_cookies:
    try:
        driver.add_cookie(cookie)
    except Exception as e:
        print(f"쿠키 추가 중 오류: {e}")

driver.refresh()
time.sleep(10)  # 로그인 세션 반영 대기

for i in range(0, total, chunk_size):
    chunk = title_ids[i:i+chunk_size]
    chunk_results = []

    for j, title_id in enumerate(chunk):
        url = f"https://page.kakao.com/content/{title_id}"
        driver.get(url)
        time.sleep(random.uniform(3, 5))

        try:
            main_title_elem = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "span.font-large3-bold"))
            )
            main_title = main_title_elem.text
            clean_main_title = re.sub(r'\[.*?\]', '', main_title).strip()

            try:
                same_work_section = driver.find_element(By.XPATH, "//div[@data-t-obj='{\"click\":{\"layer1\":\"동일작추천\"}}']")
                webnovel_elements = same_work_section.find_elements(By.XPATH, ".//li[.//span[text()='웹소설']]")
                webnovel_titles = []

                for li in webnovel_elements:
                    try:
                        title_div = li.find_element(By.CSS_SELECTOR, "div.font-small1")
                        raw_title = title_div.text
                        clean_title = re.sub(r'\[.*?\]', '', raw_title).strip()
                        webnovel_titles.append(clean_title)
                    except:
                        pass
            except:
                webnovel_titles = []

            novel_original = 1 if clean_main_title in webnovel_titles else 0

        except Exception as e:
            print(f"[{i+j+1}/{total}] title_id={title_id} ❌ 에러 발생: {e}")
            novel_original = 0

        print(f"[{i+j+1}/{total}] title_id={title_id} novel_original={novel_original}")
        chunk_results.append({"title_id": title_id, "novel_original": novel_original})

    file_num = (i // chunk_size) + 1
    output_path = os.path.join(output_folder, f"kakao_novel_original_under19_{file_num}.csv")
    pd.DataFrame(chunk_results).to_csv(output_path, index=False, encoding="utf-8-sig")
    print(f"✅ 저장 완료: {output_path} (진행률: {(i+j+1)/total:.2%})")

driver.quit()
print("🎉 전체 크롤링 완료!")


[1/363] title_id=62477346 novel_original=1
[2/363] title_id=65332544 novel_original=0
[3/363] title_id=64867816 novel_original=0
[4/363] title_id=66119536 novel_original=0
[5/363] title_id=66110723 novel_original=0
[6/363] title_id=65907692 novel_original=0
[7/363] title_id=64674813 novel_original=0
[8/363] title_id=64082520 novel_original=0
[9/363] title_id=60343223 novel_original=1
[10/363] title_id=64867866 novel_original=0
[11/363] title_id=65142916 novel_original=0
[12/363] title_id=63644867 novel_original=1
[13/363] title_id=63639661 novel_original=0
[14/363] title_id=64924117 novel_original=0
[15/363] title_id=64095113 novel_original=0
[16/363] title_id=65567376 novel_original=0
[17/363] title_id=65198662 novel_original=0
[18/363] title_id=64144969 novel_original=1
[19/363] title_id=64115859 novel_original=0
[20/363] title_id=64511192 novel_original=0
[21/363] title_id=62264793 novel_original=1
[22/363] title_id=66198911 novel_original=0
[23/363] title_id=64922461 novel_original

In [21]:
import pandas as pd
import os

folder_path = "카카오_원작 여부_only19"
file_names = [
    "kakao_novel_original_under19_1.csv",
    "kakao_novel_original_under19_2.csv",
    "kakao_novel_original_under19_3.csv",
    "kakao_novel_original_under19_4.csv"
]

dfs = []
for file in file_names:
    full_path = os.path.join(folder_path, file)
    if os.path.exists(full_path):
        df = pd.read_csv(full_path, encoding="utf-8-sig")
        dfs.append(df)
    else:
        print(f"⚠️ 파일이 존재하지 않습니다: {full_path}")

merged_df = pd.concat(dfs, ignore_index=True)
merged_df = merged_df.drop_duplicates()  # 중복 제거, 필요없으면 삭제

output_path = os.path.join(folder_path, "kakao_novel_original_under19_merged.csv")
merged_df.to_csv(output_path, index=False, encoding="utf-8-sig")

print(f"✅ 파일 병합 완료: {output_path}")


✅ 파일 병합 완료: 카카오_원작 여부_only19\kakao_novel_original_under19_merged.csv
